<a href="https://colab.research.google.com/github/napatswift/wv-th-bdgt/blob/main/example/create_validation_spreadsheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/napatswift/wv-th-bdgt

Cloning into 'wv-th-bdgt'...
remote: Enumerating objects: 190, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 190 (delta 71), reused 154 (delta 46), pack-reused 0
Receiving objects: 100% (190/190), 1.06 MiB | 19.05 MiB/s, done.
Resolving deltas: 100% (71/71), done.


In [2]:
%cd wv-th-bdgt
!pip install -r requirements.txt

/content/wv-th-bdgt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 50.0 MB/s eta 0:00:00


In [3]:
from thbud.model import BudgetItem
import pandas as pd

We can build the tree from dictionary (if it's in JSON file, read and `load` it using json library.) And then call `to_table_rows` method to generate speadsheet rows.

The method `to_table_row` will also compute the validation step and add error messsage to the column `error_message`.

In [4]:
rows = BudgetItem.from_json({
    'budget_type': 'MINISTRY',
    'name': 'ministry of magic',
    'amount': 100,
    'document': 'path/to/doc.pdf',
    'page': 1,
    'fiscal_year_budget': [],
    'children': [
        {
            'budget_type': 'BUDGETARY_UNIT',
            'name': 'this is a sample',
            'amount': 3240,
            'document': 'path/to/doc.pdf',
            'page': 1,
            'fiscal_year_budget': [{
                'year': 2556,
                'amount': 30000,
            }],
        }
    ],
}).to_rows()

Read from the file

In [5]:
import json

with open('/content/budget-1page-5nodes.json') as f:
    rows = BudgetItem.from_json(json.load(f)).to_rows()

After we've created the rows from the tree, we can now build the dataframe using pandas.

In [8]:
df = pd.DataFrame(rows)
name_cols = [col for col in df.columns if col.startswith('name_')]
name_cols.sort()
cols = ['error_message', 'budget_type', *name_cols,]
if 'fiscal_year' in name_cols:
    cols += [ 'fiscal_year', 'fiscal_year_end',]
cols += ['amount', 'document','page']

df = df[cols]

In [11]:
df

,error_message,budget_type,name_1,name_2,name_3,name_4,name_5,name_6,name_7,name_8,amount,document,page
0,While checking sum: amount of ROOT is None but...,ROOT,ROOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,0
1,,BUDGET_PLAN,NaN,"7.1 แ�นงานบุคลากรภาครัฐ 109,849,900 บาท",NaN,NaN,NaN,NaN,NaN,NaN,109849900.0,,0
2,,BUDGET_DETAIL,NaN,NaN,"1. งบเงินอุดหนุน 109,849,900 บาท",NaN,NaN,NaN,NaN,NaN,109849900.0,,1
3,,BUDGET_DETAIL,NaN,NaN,NaN,"1.1 เงินอุดหนุนทั่วไป 109,849,900 บาท",NaN,NaN,NaN,NaN,109849900.0,,1
4,,BUDGET_DETAIL,NaN,NaN,NaN,NaN,"1) ค่าใช้จ่ายบุคลากร 98,082,900 บาท",NaN,NaN,NaN,98082900.0,,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,,BUDGET_DETAIL,NaN,NaN,NaN,NaN,NaN,NaN,"1.1.2.1 ค่า�รับ�รุงสิ�งก่อสร้างอ��น 2,985,000 บาท",NaN,2985000.0,,11
82,,BUDGET_DETAIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(1) ค่า�รับ�รุงสิ�งก่อสร้างอ��นที�มีราคา�่อ�น่...,2985000.0,,11
83,,BUDGET_DETAIL,NaN,NaN,NaN,"2. งบเงินอุดหนุน 238,958,600 บาท",NaN,NaN,NaN,NaN,238958600.0,,11
84,,BUDGET_DETAIL,NaN,NaN,NaN,NaN,"2.1 เงินอุดหนุนทั่วไป 238,958,600 บาท",NaN,NaN,NaN,238958600.0,,11


We can save the dataframe to storage and validate later.